# Reranking trong RAG
## Lost In The Middle
Khi LLM nhận một context quá dài và thông tin cần thiết, có liên quan đến câu hỏi nằm ở phần giữa của context đó, thì nó sẽ có xu hướng là bỏ qua văn bản đó.

Nên để giải quyết việc này, cần sắp xếp thông tin có liên quan ở phía đầu và phía cuối của context (nơi mà LLM sẽ tập trung vào nhiều hơn), và đưa thông tin không liên quan vào giữa context.

In [ ]:
! pip install cohere

In [6]:
from langchain_community.document_transformers import (
    LongContextReorder,
)
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.storage import InMemoryStore
from langchain.retrievers.document_compressors import CohereRerank
import os
from dotenv import load_dotenv

In [11]:
load_dotenv()
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

In [2]:
HF_EMBEDDING = HuggingFaceEmbeddings(model_name="keepitreal/vietnamese-sbert")
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1,
    chunk_overlap=0,
)

vectorstore = Chroma(persist_directory="./data" ,embedding_function=HF_EMBEDDING)

/home/tranhuy/Desktop/Project/cantho-public-services/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
vectorstore.similarity_search()

In [3]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
question = "thủ tục đăng ký lại giấy chứng sinh cho con cần làm giấy tờ gì không?"
docs = retriever.get_relevant_documents(question)
docs

[Document(page_content='Cần chuẩn bị các hồ sơ sau đây và nộp tại Bộ phận Tiếp nhận và Trả kết quả của Ủy ban nhân dân cấp xã với thủ tục hành chính Thủ tục về Đăng ký khai sinh và cấp thẻ bảo hiểm y tế (cấp mới, cấp đổi và cấp lại) cho trẻ em dưới 6 tuổi:\n- Ghi chú: * Hồ sơ phải nộp:\n2. + Giấy chứng sinh. Trường hợp không có giấy chứng sinh thì nộp văn bản của người làm chứng xác nhận về việc sinh; nếu không có người làm chứng thì phải có Giấy cam đoan về việc sinh; trường hợp khai sinh cho trẻ em bị bỏ rơi phải có biên bản xác nhận việc trẻ bị bỏ rơi do cơ  quan có thẩm quyền lập; trường hợp khai sinh cho trẻ em sinh ra do mang thai hộ  phải có văn bản xác nhận của cơ sở y tế đã thực hiện kỹ thuật hỗ trợ sinh sản cho việc mang thai hộ. Văn bản ủy quyền theo quy định của pháp luật trong trường hợp ủy quyền thực hiện việc đăng ký khai sinh. Trường hợp người được ủy quyền là ông, bà, cha, mẹ, con, vợ, chồng, anh, chị, em ruột của người ủy quyền thì văn bản ủy quyền không phải công chứ

In [4]:
tokens = 0
for doc in docs:
    tokens += len(doc.page_content.split(" "))
tokens

6881

In [5]:
reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs)
reordered_docs

[Document(page_content='Cần chuẩn bị các hồ sơ sau đây và nộp tại Bộ phận Tiếp nhận và Trả kết quả của UBND cấp xã với thủ tục hành chính Thủ tục về Đăng ký khai sinh, đăng ký thường trú, cấp thẻ bảo hiểm y tế (cấp mới, cấp đổi và cấp lại) cho trẻ em dưới 6 tuổi:\n- Ghi chú: * Hồ sơ phải nộp:\n2. + Giấy chứng sinh. Trường hợp không có Giấy chứng sinh thì nộp văn bản của người làm chứng xác nhận về việc sinh; nếu không có người làm chứng thì phải có Giấy cam đoan về việc sinh; trường hợp khai sinh cho trẻ em bị bỏ rơi phải có biên bản xác nhận việc trẻ bị bỏ rơi do cơ quan có thẩm quyền lập; trường hợp khai sinh cho trẻ em sinh ra do mang thai hộ phải có văn bản xác nhận của cơ sở y tế đã thực hiện kỹ thuật hỗ trợ sinh sản cho việc mang thai hộ;  Văn bản ủy quyền theo quy định của pháp luật trong trường hợp ủy quyền thực hiện việc đăng ký khai sinh. Trường hợp người được ủy quyền là ông, bà, cha, mẹ, con, vợ, chồng, anh, chị, em ruột của người ủy quyền thì văn bản ủy quyền không phải cô

In [ ]:
import cohere 

co = cohere.Client(api_key=COHERE_API_KEY)

docs = [reordered_doc.page_content for reordered_doc in reordered_docs]
results = co.rerank(model="rerank-multilingual-v2.0", query=question, documents=docs, top_n=3)

In [10]:
for result in results:
    print(result.document)

{'text': 'Cần chuẩn bị các hồ sơ sau đây và nộp tại ủy ban nhân dân cấp xã với thủ tục hành chính Thủ tục đăng ký khai sinh:\n- Ghi chú: * Giấy tờ phải nộp:\n2. - Bản chính Giấy chứng sinh; trường hợp không có Giấy chứng sinh thì nộp văn bản của người làm chứng xác nhận về việc sinh; nếu không có người làm chứng thì phải có giấy cam đoan về việc sinh;: cần 1 bản chính và 0 bản sao.. Mẫu đơn cho hồ sơ này có thể tìm thấy tại \n3. - Trường hợp trẻ em bị bỏ rơi thì phải có biên bản về việc trẻ bị bỏ rơi do cơ quan có thẩm quyền lập.: cần 1 bản chính và 0 bản sao.. Mẫu đơn cho hồ sơ này có thể tìm thấy tại \n4. - Văn bản ủy quyền (được chứng thực) theo quy định của pháp luật trong trường hợp ủy quyền thực hiện việc đăng ký khai sinh.: cần 1 bản chính và 0 bản sao.. Mẫu đơn cho hồ sơ này có thể tìm thấy tại \n- Ghi chú: \n6. - Mẫu hộ tịch điện tử tương tác đăng ký khai sinh (do người yêu cầu cung cấp thông tin theo hướng dẫn trên Cổng dịch vụ công, nếu người có yêu cầu lựa chọn nộp hồ sơ th